In [16]:
import os, geopandas as gpd, networkx as nx
from shapely.geometry import Point, MultiLineString, LineString
from networkx import Graph, connected_components
import warnings
warnings.filterwarnings("ignore")

In [2]:
subdivision_dataset = gpd.read_file("../../../../../Edited Subdivisions Zurikanen Erfan 4-20=2025/subs_plus_new/sub.gdb",layer='subs_new_combined_4_18_2025_2010_2023')
source_edges_path = "../../../Data/Original_dataset/Archive/centerlinemeck_10_8/"
source_edges = gpd.read_file("../../../Data/Original_dataset/Archive/centerlinemeck_10_8/edges_version_2.shp")

In [3]:
#1 Node extraction

nodes_version_1_path = os.path.join(source_edges_path, "nodes.shp")

if __name__ == "__main__":

    # Create an empty list to store the endpoint coordinates
    endpoint_coords = []
    # Iterate through the street network and find endpoint coordinates
    node_id = 0; node_id_mapping = {}
    
   
    # Iterate through the edges and find the end points coordinates
    for index, row in source_edges.iterrows():
        geometry = row['geometry']
        if geometry.geom_type == 'MultiLineString':
             print(f"Row {index} has a MultiLineString; using the first LineString component.")
             continue           
        start_point = Point(geometry.coords[0])
        end_point = Point(geometry.coords[-1])

        # Add start and end node ids to the edge GeoDataFrame
        if start_point.coords[0] not in node_id_mapping:
            endpoint_coords.append({'id': node_id, 'geometry': start_point})
            node_id_mapping[start_point.coords[0]] = node_id
            start_node_id = node_id
            node_id += 1
        else:
            start_node_id = node_id_mapping[start_point.coords[0]]

        if end_point.coords[0] not in node_id_mapping:
            endpoint_coords.append({'id': node_id, 'geometry': end_point})
            node_id_mapping[end_point.coords[0]] = node_id
            end_node_id = node_id
            node_id += 1
        else:
            end_node_id = node_id_mapping[end_point.coords[0]]

        #Adding corresponding start and end node's ID to each edge
        source_edges.at[index, 'start_node'] = start_node_id
        source_edges.at[index, 'end_node'] = end_node_id


        print(f"The edge {index} has been merged successfully!")

 
    # Create a GeoDataFrame from the endpoint coordinates
    source_nodes_gdf = gpd.GeoDataFrame(endpoint_coords, crs=source_edges.crs)
    source_nodes_gdf['x'] = source_nodes_gdf['geometry'].x; source_nodes_gdf['y'] = source_nodes_gdf['geometry'].y; source_nodes_gdf["osmid"] = source_nodes_gdf["id"].astype("int")
    source_nodes_gdf.set_index('id', inplace=True)  # Set 'id' column as index

    # Save the node shapefile and indexed edges
    source_nodes_gdf.to_file(nodes_version_1_path)
    
    #Exporting updated edges
    source_edges_path_updated = os.path.join(source_edges_path, "edges_updated.shp")
    source_edges.to_file(source_edges_path_updated)

    print(f"The nodes of the source edgee has been created successfully!")


The edge 0 has been merged successfully!
The edge 1 has been merged successfully!
The edge 2 has been merged successfully!
The edge 3 has been merged successfully!
The edge 4 has been merged successfully!
The edge 5 has been merged successfully!
The edge 6 has been merged successfully!
The edge 7 has been merged successfully!
The edge 8 has been merged successfully!
The edge 9 has been merged successfully!
The edge 10 has been merged successfully!
The edge 11 has been merged successfully!
The edge 12 has been merged successfully!
The edge 13 has been merged successfully!
The edge 14 has been merged successfully!
The edge 15 has been merged successfully!
The edge 16 has been merged successfully!
The edge 17 has been merged successfully!
The edge 18 has been merged successfully!
The edge 19 has been merged successfully!
The edge 20 has been merged successfully!
The edge 21 has been merged successfully!
The edge 22 has been merged successfully!
The edge 23 has been merged successfully!
Th

In [11]:
#Local networks

edges_version_2_path = source_edges_path_updated
edges_version_2 = gpd.read_file(edges_version_2_path) #Calling rectified edges
edges_version_2.reset_index(drop=True, inplace=True); edges_version_2 = edges_version_2.drop(columns=['start_node', 'end_node']) #Resetting source edge's index and removing unnecessary columns

def convert_multilinestring_to_linestring(geom):
    """
    Converts a MultiLineString geometry into a single LineString.
    If the geometry is already a LineString, it returns it unchanged.
    """
    if isinstance(geom, MultiLineString):
        # Collect all coordinates from the MultiLineString
        coords = []
        for part in geom.geoms:
            coords.extend(part.coords)
        # Create a new LineString from the concatenated coordinates
        return LineString(coords)
    else:
        # If it's already a LineString, return it as is
        return geom
    
edges_version_2['geometry'] = edges_version_2['geometry'].apply(convert_multilinestring_to_linestring) # Apply the conversion to an entire GeoDataFrame column

NPA_shape = subdivision_dataset

buffer_list_mile = [0.1,0.25,0.5,0.75,1.0]
buffer_distance_feet = []

for buffer_mile in buffer_list_mile:
    buffer_distance_feet.append(buffer_mile * 5280)
    buffer_mile_name = str(buffer_mile).replace(".", "")
    NPA_shape['geometry'] = NPA_shape['geometry'].apply(lambda geom: geom.buffer(buffer_mile * 5280)) #Applying buffer operation to the geometry column


    for idx, polygon in NPA_shape.iterrows():
        
        edge_NPA = edges_version_2[edges_version_2.intersects(polygon.geometry)]     # Find edges intersecting with the buffered polygon
    
        #Extracting nodes at the end of each edge
    
        endpoint_coords = []     # Create an empty list to store the endpoint coordinates
        node_id = 0     # Iterate through the street network and find endpoint coordinates
        node_id_mapping = {}
    
        # Iterate through the street network and find endpoint coordinates
        for index, row in edge_NPA.iterrows():
            geometry = row['geometry']
    
            # Check if the geometry is single-part or multi-part
            if geometry.geom_type == 'LineString':
                # Single-part geometry, safe to access coords directly
                start_point = Point(geometry.coords[0])
                end_point = Point(geometry.coords[-1])
            elif geometry.geom_type == 'MultiLineString':
                # Multi-part geometry: get the start point of the first part and the end point of the last part
                first_part = geometry.geoms[0]  # First LineString
                last_part = geometry.geoms[-1]  # Last LineString
                start_point = Point(first_part.coords[0])  # Start of first part
                end_point = Point(last_part.coords[-1])
    
            # Add start and end node ids to the edge GeoDataFrame
            if start_point.coords[0] not in node_id_mapping:
                endpoint_coords.append({'id': node_id, 'geometry': start_point})
                node_id_mapping[start_point.coords[0]] = node_id
                start_node_id = node_id
                node_id += 1
            else:
                start_node_id = node_id_mapping[start_point.coords[0]]
    
            if end_point.coords[0] not in node_id_mapping:
                endpoint_coords.append({'id': node_id, 'geometry': end_point})
                node_id_mapping[end_point.coords[0]] = node_id
                end_node_id = node_id
                node_id += 1
            else:
                end_node_id = node_id_mapping[end_point.coords[0]]
    
            edge_NPA.at[index, 'start_node'] = start_node_id
            edge_NPA.at[index, 'end_node'] = end_node_id
               
        # Create a GeoDataFrame from the endpoint coordinates
        nodes_NPA = gpd.GeoDataFrame(endpoint_coords, crs=edge_NPA.crs)
        nodes_NPA['x'] = nodes_NPA['geometry'].x
        nodes_NPA['y'] = nodes_NPA['geometry'].y
        nodes_NPA["osmid"] = nodes_NPA["id"].astype("int")
        nodes_NPA.set_index('id', inplace=True)  # Set 'id' column as index
    
        #indexing NPA edges
        edge_NPA["u"] = edge_NPA["start_node"].astype("int")
        edge_NPA["v"] = edge_NPA["end_node"].astype("int")
        edge_NPA["k"] = range(1, len(edge_NPA) + 1)
        edge_NPA["osmid"] = edge_NPA["k"]
        edge_NPA.set_index(["u","v","k"], inplace=True)
        # print(f"network elements for {idx} has been created successfully!")

        # Define the directory path
        output_dir = os.path.join(source_edges_path, f"local_grpahs/{polygon['subd_id']}_{buffer_mile_name}"); os.makedirs(output_dir, exist_ok=True)

        # Save the GeoDataFrame to a shapefile
        nodes_version_3_path = os.path.join(output_dir, f"nodes_version_3.shp")
        edges_version_3_path = os.path.join(output_dir, f"edges_version_3.shp")
        nodes_NPA.to_file(nodes_version_3_path); edge_NPA.to_file(edges_version_3_path)
        print(f"local network has been created for the local graph {polygon['subd_id']}_{buffer_mile_name}")

    NPA_shape = subdivision_dataset #reseting the subdivision dataset

local network has been created for the local graph 95.0_01
local network has been created for the local graph 147.0_01
local network has been created for the local graph 185.0_01
local network has been created for the local graph 200.0_01
local network has been created for the local graph 212.0_01
local network has been created for the local graph 243.0_01
local network has been created for the local graph 249.0_01
local network has been created for the local graph 261.0_01
local network has been created for the local graph 272.0_01
local network has been created for the local graph 302.0_01
local network has been created for the local graph 320.0_01
local network has been created for the local graph 328.0_01
local network has been created for the local graph 353.0_01
local network has been created for the local graph 364.0_01
local network has been created for the local graph 367.0_01
local network has been created for the local graph 405.0_01
local network has been created for the lo

KeyboardInterrupt: 

In [15]:
#extracting the largest network
for idx, polygon in NPA_shape.iterrows():

    output_dir = os.path.join(source_edges_path, f"local_grpahs/{polygon['subd_id']}_{buffer_mile_name}")
    nodes_version_3_path = os.path.join(output_dir, f"nodes_version_3.shp")
    edges_version_3_path = os.path.join(output_dir, f"edges_version_3.shp")
    edge_version_3 = gpd.read_file(edges_version_3_path)
    nodes_version_3 = gpd.read_file(nodes_version_3_path)


    # remove any unconnected node and edge to the main network
    G = Graph()

    # Add nodes from nodes GeoDataFrame
    for node_id, geometry in zip(nodes_version_3['osmid'], nodes_version_3['geometry']):
        G.add_node(node_id, pos=(geometry.x, geometry.y))  # Store node positions if needed

    # Add edges from edges GeoDataFrame
    for source, target, geometry in zip(edge_version_3['start_node'], edge_version_3['end_node'], edge_version_3['geometry']):
        G.add_edge(source, target, geometry=geometry)

    #getting the largest part of the network
    components = list(connected_components(G))
    largest_component = max(components, key=len)
    G_main = G.subgraph(largest_component).copy()

    # appending the overpasses and underpasses to the main network
    remaining_components = [component for component in components if component != largest_component] # Store the remaining components (all except the largest)
    # Convert edges of the largest component to a GeoDataFrame for spatial operations
    edges_main_gdf = edge_version_3[edge_version_3['start_node'].isin(largest_component) & edge_version_3['end_node'].isin(largest_component)]

    # Loop through each remaining component
    for component in remaining_components:
        # Extract edges of the current component
        edges_component_gdf = edge_version_3[(edge_version_3['start_node'].isin(component)) & (edge_version_3['end_node'].isin(component))]
        # Check if any edges intersect with the largest component's edges
        if edges_component_gdf.geometry.intersects(edges_main_gdf.geometry.unary_union).any():
            # If they intersect, add this component to the largest component
            for node in component:
                G_main.add_node(node)  # Add nodes of the component
            for _, edge in edges_component_gdf.iterrows():
                G_main.add_edge(edge['start_node'], edge['end_node'], geometry=edge['geometry'])  # Add edges


    # Get the nodes that are part of the main component
    main_nodes = list(G_main.nodes())
    # Filter nodes_gdf to keep only nodes in the main component
    nodes_gdf_main = nodes_version_3[nodes_version_3['osmid'].isin(main_nodes)].copy()

    # Export the edeges of G_main to GeoDataFrame
    main_edges = list(G_main.edges(data=True))  # Get edges with data (geometry)
    if main_edges:  # Check if there are edges in G_main
        # Create a list to hold edge data
        edges_data = []

        for (start, end, data) in main_edges:
            edges_data.append({
                'start_node': start,
                'end_node': end,
                'geometry': data['geometry']
            })

        # Create a GeoDataFrame from the list of edges data
        gdf_edges_main = gpd.GeoDataFrame(edges_data)

        # Set CRS if known
        gdf_edges_main.set_crs(edge_version_3.crs, inplace=True)

        # Optionally save the GeoDataFrame to a shapefile
        edge_version_4_path = os.path.join(os.path.dirname(edges_version_3_path), f"edges_version_4.shp")
        gdf_edges_main.to_file(edge_version_4_path)
        print("Saved edges of the largest component to shapefile.")
    else:
        print("G_main has no edges to convert to GeoDataFrame.")

    # Export the nodes of G_main to GeoDataFrame
    main_nodes = list(G_main.nodes(data=True))  # Get nodes with data (geometry)
    if main_nodes:  # Check if there are nodes in G_main
        # Create a list to hold node data
        nodes_data = []

        for node, data in main_nodes:
            # Get the geometry for the node
            node_geometry = nodes_version_3[nodes_version_3['osmid'] == node]['geometry'].values[0]
            nodes_data.append({
                'osmid': node,
                'geometry': node_geometry
            })

        # Create a GeoDataFrame from the list of nodes data
        gdf_nodes_main = gpd.GeoDataFrame(nodes_data)

        # Set CRS if known
        gdf_nodes_main.set_crs(nodes_version_3.crs, inplace=True)

        # Optionally save the GeoDataFrame to a shapefile
        nodes_version_4_path = os.path.join(os.path.dirname(nodes_version_3_path), f"nodes_version_4.shp")
        gdf_nodes_main.to_file(nodes_version_4_path)
        print("Saved nodes of the largest component to shapefile.")
    else:
        print("G_main has no nodes to convert to GeoDataFrame.")

Saved edges of the largest component to shapefile.
Saved nodes of the largest component to shapefile.
Saved edges of the largest component to shapefile.
Saved nodes of the largest component to shapefile.
Saved edges of the largest component to shapefile.
Saved nodes of the largest component to shapefile.
Saved edges of the largest component to shapefile.
Saved nodes of the largest component to shapefile.
Saved edges of the largest component to shapefile.
Saved nodes of the largest component to shapefile.
Saved edges of the largest component to shapefile.
Saved nodes of the largest component to shapefile.
Saved edges of the largest component to shapefile.
Saved nodes of the largest component to shapefile.
Saved edges of the largest component to shapefile.
Saved nodes of the largest component to shapefile.
Saved edges of the largest component to shapefile.
Saved nodes of the largest component to shapefile.
Saved edges of the largest component to shapefile.
Saved nodes of the largest comp

DataSourceError: ../../../Data/Original_dataset/Archive/centerlinemeck_10_8/local_grpahs/950.0_01\edges_version_3.shp: No such file or directory

In [ ]:
for buffer_mile in buffer_list_mile:
    
    buffer_mile_name = str(buffer_mile).replace(".", "")

    for idx, polygon in NPA_shape.iterrows():
        
        output_dir = os.path.join(source_edges_path, f"local_grpahs/{polygon['id']}_{buffer_mile_name}")
        edges_version_4_1_path = os.path.join(output_dir, f"edges_version_4_1.shp")
        edges_version_4_1 = gpd.read_file(edges_version_4_1_path,index=False); edges_version_4_1 = edges_version_4_1.set_crs(crs=NPA_shape.crs)
        nodes_version_4_1_path = os.path.join(output_dir, f"nodes_version_4_1.shp")
        nodes_version_4_1 = gpd.read_file(nodes_version_4_1_path, index=False); nodes_version_4_1 = nodes_version_4_1.set_crs(crs=NPA_shape.crs)
    
        edges_version_4_1["u"] = edges_version_4_1["start_node"].astype("int"); edges_version_4_1["v"] = edges_version_4_1["end_node"].astype("int")
        edges_version_4_1["key"] = range(1, len(edges_version_4_1) + 1)
        edges_version_4_1["osmid"] = edges_version_4_1["key"]
        edges_version_4_1.set_index(["u","v","key"], inplace=True)
        edges_version_4_1['length'] = edges_version_4_1['geometry'].length
        
        # Initialize a directed or undirected graph using nx
        G = nx.Graph()
        
        # Add nodes with attributes
        for _, row in nodes_version_4_1.iterrows():
            node_id = row['osmid']  # Adjust column name if needed
            G.add_node(node_id, **row.to_dict())
    
        # Add edges with attributes
        for _, row in edges_version_4_1.iterrows():
            start_node = int(row['start_node'])  # Adjust column names if needed
            end_node = int(row['end_node'])
            edge_data = {
                'length': row['geometry'].length,
                # Add other edge attributes if necessary
            }
            G.add_edge(start_node, end_node, **edge_data, weight = row['geometry'].length)
    
        #converting to undirected graph
        G = G.to_undirected()
        
        #area of interest
        temp_gdf = gpd.GeoDataFrame(geometry=[polygon.geometry], crs=NPA_shape.crs)
        temp_gdf['geometry'] = temp_gdf['geometry'].apply(lambda geom: geom.buffer(buffer_mile * 5280))
        temp_buffer = temp_gdf["geometry"].area[0]
        area_acre = temp_buffer / 43560
    
    
        # intersection density
        intersections = [node for node in G.nodes if G.degree(node) >= 3]; num_intersections = len(intersections); intersection_density = num_intersections / area_acre
        if "int_den" + buffer_mile_name not in NPA_shape.columns:
            NPA_shape["int_den" + buffer_mile_name] = 0
        NPA_shape.loc[idx, "int_den"+ buffer_mile_name] = intersection_density
        
        # avg. node degree
        average_node_degree = sum(dict(G.degree()).values()) / G.number_of_nodes(); NPA_shape.loc[idx, "nd_deg" + buffer_mile_name] = average_node_degree